# Global Sertlik Matrisi ve Kuvvet Vektörünün Oluşturulması

Geçtiğimiz bölümde eleman sertlik matrisini ve kuvvet vektörünü referans elemanlar için integral formunda türetmiştik. Şimdi bu türettiğimiz denklemleri nümerik olarak hesaplayıp, global sertlik matrisini ve kuvvet vektörünü elde edelim. Hesaplamalara başlamadan önce problemin çözümünde kullanacağımız ağı (mesh) tanımlayalım. Şekil 3'te kullanacağımız ağ yapısı gösterilmiştir. Gri noktalar düğüm noktalarını göstermektedir. Eleman numaraları elemanların içinde yuvarlak içerisinde gösterilirken, düğüm noktalarının yanında numaraları ve $x-y$ koordinat sisteminde konumları metre cinsinden gösterilmektedir. 

<img src="Resimler/Ders8/Sekil3.png" align="center" width= "700"/>

## Global Sertlik Matrisinin Oluşturulması

Öncelikle 8b'de referans eleman için türettiğimiz yerel sertlik matrisini ifade eden integrali hatırlayalım:

$$\int_{-1}^{1}\int_{-1}^{1} \mathbf{N}^{T}\mathbf{J}^{T}\mathbf{H}^{T}\mathbf{C}\mathbf{H}\mathbf{J}\mathbf{N}d\eta d\xi det(\mathbf{J}).\tag{1} $$


Şimdi de integral içerisindeki matrisleri hatırlayalım:

$$ \mathbf{C}= \frac{E}{(1-\nu^{2})} \begin{bmatrix} 1 & \nu & 0 \\ \nu & 1 & 0 \\ 0 & 0 & \frac{1-\nu}{2} \end{bmatrix}, $$
$$ \mathbf{H}=\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 1 & 1 & 0 \end{bmatrix},$$
$$\mathbf{J}=\begin{bmatrix} \mathbf{J^{-1}} & 0 \\ 0 & \mathbf{J^{-1}} \end{bmatrix},$$
$$\mathbf{N}=\begin{bmatrix} \frac{\partial N_1}{\partial \xi}  & 0 & \frac{\partial N_2}{\partial \xi}  & 0 & \frac{\partial N_3}{\partial \xi} & 0 & \frac{\partial N_4}{\partial \xi} & 0 & \\ \\ \frac{\partial N_1}{\partial \eta} & 0  & \frac{\partial N_2}{\partial \eta} & 0  & \frac{\partial N_3}{\partial \eta} & 0 & \frac{\partial N_4}{\partial \eta} & 0 \\ \\ 0  & \frac{\partial N_1}{\partial \xi}  & 0 & \frac{\partial N_2}{\partial \xi} & 0  & \frac{\partial N_3}{\partial \xi}  & 0 & \frac{\partial N_4}{\partial \xi}\\ \\ 0  & \frac{\partial N_1}{\partial \eta}  & 0 & \frac{\partial N_2}{\partial \eta} & 0  & \frac{\partial N_3}{\partial \eta}  & 0 & \frac{\partial N_4}{\partial \eta} \end{bmatrix}.$$

Görülebileceği üzere $\mathbf{C}$ ve $\mathbf{H}$ sabit değerli matrislerdir. İntegrali hesaplayabilmek için $\mathbf{J}$ ve $\mathbf{N}$ matirslerini tanımlamamız gerekir. Öncelikle bunun için 7. bölümde olduğu gibi *sympy* kütüphanesini kullanalım. 

In [1]:
import sys

$\mathbf{N}$ matrisini yazabilmek için şekil fonksiyonlarını sembolik olarak yazalım:

In [2]:
import sympy as sp
import numpy as np

xi = sp.Symbol("xi")
eta = sp.Symbol("eta")
x_ = sp.symbols("x1,x2,x3,x4")
y_ = sp.symbols("y1,y2,y3,y4")
şekil = sp.Matrix([(1+xi)*(1+eta)/4,
                   (1-xi)*(1+eta)/4,
                   (1-xi)*(1-eta)/4,
                   (1+xi)*(1-eta)/4])
şekil

Matrix([
[(eta + 1)*(xi + 1)/4],
[(1 - xi)*(eta + 1)/4],
[(1 - eta)*(1 - xi)/4],
[(1 - eta)*(xi + 1)/4]])

Şimdi şekil fonksiyonlarının $\xi$ ve $\eta$'e göre türevlerini alıp $\mathbf{N}$'i tanımlayalım:

In [3]:
gradyen_xi = sp.diff(şekil, xi)
gradyen_eta = sp.diff(şekil, eta)

N = sp.Matrix([[gradyen_xi[0],0,gradyen_xi[1],0,gradyen_xi[2],0, gradyen_xi[3],0],[gradyen_eta[0],0,gradyen_eta[1],0,gradyen_eta[2],0, gradyen_eta[3],0],[0,gradyen_xi[0],0,gradyen_xi[1],0, gradyen_xi[2],0,gradyen_xi[3]],[0,gradyen_eta[0],0,gradyen_eta[1],0, gradyen_eta[2],0,gradyen_eta[3]]])

N

Matrix([
[eta/4 + 1/4,           0, -eta/4 - 1/4,            0, eta/4 - 1/4,           0, 1/4 - eta/4,           0],
[ xi/4 + 1/4,           0,   1/4 - xi/4,            0,  xi/4 - 1/4,           0, -xi/4 - 1/4,           0],
[          0, eta/4 + 1/4,            0, -eta/4 - 1/4,           0, eta/4 - 1/4,           0, 1/4 - eta/4],
[          0,  xi/4 + 1/4,            0,   1/4 - xi/4,           0,  xi/4 - 1/4,           0, -xi/4 - 1/4]])

Şimdi jakopyen matrisini oluşturup 1. eleman için $\mathbf{J}$ matrisini hesaplayalım:

In [29]:
c_şekil = sp.Matrix([0, 0])
for i in range(4):
    c_şekil[0] += x_[i] * şekil[i]
    c_şekil[1] += y_[i] * şekil[i]
j = c_şekil.jacobian([xi, eta]).T
x_koord = [0,     0.16,    0.16,  0.0] 
y_koord = [-0.32,-0.20,   -0.1, -0.16]
    
for nokta in range(4):
    j = j.subs(x_[nokta],x_koord[nokta]) 
    j = j.subs(y_[nokta],y_koord[nokta])

J= sp.Matrix.zeros(4,4)
J[:2,:2] = j.inv()
J[2:,2:] = j.inv()
det = j.det()


0.0012*xi + 0.0052

Son olarak $\mathbf{C}$ ve $\mathbf{H}$ matrislerini tanımlayıp, 1. eleman için eleman sertlik matrisini elde edelim:

In [31]:
E=10000000
v=0.3
H=sp.Matrix([[1,0,0,0],[0,0,0,1],[0,1,1,0]])
C=sp.Matrix([[1,v,0],[v,1,0],[0,0,(1-v)/2]])*E/(1-v**2)
K_ele_sys=N.T*J.T*H.T*C*H*J*N*det
from scipy import integrate
from sympy.utilities import lambdify

K_e = np.zeros((8,8))
for i in range(8):
    for j in range(8):
        # scipy'da bulunan integrasyon fonksiyonunu kullanmak için
        # hücre değerlerini geçici olarak 'lambda' formatına çevirmemiz gerekiyor.
        geçici_fonksiyon = lambdify( (xi,eta), K_ele_sys[i,j], 'math' )
        K_e[i,j] += integrate.nquad(geçici_fonksiyon, [[-1, 1],[-1, 1]])[0]
K_e


array([[ 2302296.38439272,   297511.03435311, -1430926.96228108,
         -970523.14947047,    57300.5886547 ,  -952553.77360645,
         -928670.01076633,  1625565.88872382],
       [  297511.03435311,  4377491.53780427,  -695797.87474519,
         1498760.79226041,  -952553.77360646, -1979530.02302965,
         1350840.61399854, -3896722.30703504],
       [-1430926.96228108,  -695797.87474519,  9899373.24953983,
        -3666943.18062747, -2344428.19459479,  1743866.25755055,
        -6124018.09266397,  2618874.79782212],
       [ -970523.14947047,  1498760.79226041, -3666943.18062747,
         7979729.98513059,  2018591.53227583, -5335499.21589982,
         2618874.79782212, -4142991.56149119],
       [   57300.5886547 ,  -952553.77360645, -2344428.19459478,
         2018591.53227582,  3718054.56822117,   -95514.6091989 ,
        -1430926.96228108,  -970523.14947047],
       [ -952553.77360646, -1979530.02302965,  1743866.25755055,
        -5335499.21589981,   -95514.6091989 ,  581

Şimdi herhangi bir eleman için $\mathbf{J}$ ve eleman sertlik matrisini hesaplayan fonksiyonumuzu tanımlayalım:

In [ ]:
def lokal_jakobyen(self, x_koord,y_koord):
    # Elemana ait noktaların global indislerini içeren dizi
    eleman_nokta_indisler = self.mesh.elemanlar[eleman_no]
    # Elemana ait koordinatların x ve y noktalarını içeren dizi
    eleman_noktalar = np.take(self.mesh.noktalar, eleman_nokta_indisler, axis=0)
        
    J = self.J
    c_şekil = sp.Matrix([0, 0])
    for i in range(4):
        c_şekil[0] += x_[i] * şekil[i]
        c_şekil[1] += y_[i] * şekil[i]
        j = c_şekil.jacobian([xi, eta]).T
    
    for nokta in range(4):
        j = j.subs(x_[nokta],x_koord[nokta]) 
        j = j.subs(y_[nokta],y_koord[nokta])

        J= sp.Matrix.zeros(4,4)
        J[:2,:2] = j.inv()
        J[2:,2:] = j.inv()
        det = j.det()
        
return J


def K_e_eleman(V, eleman_no, k):

    J = V.lokal_jakobyen(eleman_no)

    K_e_sym = V.gradyen.T*(J**(-1)).T*k*(J**(-1))*V.gradyen*J.det()

    K_e = np.zeros((V.nokta_sayısı,V.nokta_sayısı))

    for i in range(V.nokta_sayısı):
        for j in range(V.nokta_sayısı):
            geçici_fonksiyon = lambdify( (V.xi,V.eta), K_e_sym[i,j], 'math' )
            K_e[i,j] += integrate.nquad(geçici_fonksiyon, [[-1, 1],[-1, 1]])[0]
            
return K_e